
## Setup AI 


In [ ]:
#! pip install tensorflow

In [ ]:
! pip install keras

In [ ]:
! pip install opencv-python

In [ ]:
pip install --upgrade pip

In [ ]:
! pip install --upgrade pip

In [ ]:
import requests
from bs4 import BeautifulSoup
res = requests.get('https://www.google.com.tw/search?ei=WchSW_n6H4Gg-Qb815eQBA&yv=3&q=%E6%8A%BD%E7%85%99&safe=off&tbm=isch&vet=10ahUKEwi5i-DMvq_cAhUBUN4KHfzrBUIQuT0IuAEoAQ.WchSW_n6H4Gg-Qb815eQBA.i&ved=0ahUKEwi5i-DMvq_cAhUBUN4KHfzrBUIQuT0IuAEoAQ&ijn=3&start=300&asearch=ichunk&async=_id:rg_s,_pms:s,_fmt:pc')
soup = BeautifulSoup(res.text, 'lxml') #lxml解析器


In [ ]:
#print(res.text)

In [ ]:
import json
images = []
for rec in soup.select('.rg_meta'):
    jd = json.loads(rec.text)
    print(jd.get('tu'))
    images.append(jd.get('tu'))

In [ ]:
len(images)

In [ ]:
import os
import json, requests
from bs4 import BeautifulSoup
#抽煙
google_url = 'https://www.google.com.tw/search?ei=l0VYW77rAsPk-AaboofoDQ&safe=off&yv=3&q={}&tbm=isch&vet=10ahUKEwj-kvDe-rncAhVDMt4KHRvRAd0QuT0IuwEoAQ.l0VYW77rAsPk-AaboofoDQ.i&ved=0ahUKEwj-kvDe-rncAhVDMt4KHRvRAd0QuT0IuwEoAQ&ijn=1&start={}&asearch=ichunk&async=_id:rg_s,_pms:s,_fmt:pc'



def getImage(keyword, image_dir, batch):    
    for ins in range(batch):
        res = requests.get(google_url.format(keyword, ins * 100))
        soup = BeautifulSoup(res.text, 'lxml') 
        
        if (os.path.exists('pic_'+image_dir + str(ins))):
            
            print("File exists!")
        else:
  
            print("patch not found,Create it  %s!", image_dir + str(ins))
            os.mkdir('pic_'+image_dir + str(ins))
        
        for rec in soup.select('.rg_meta'):
            jd = json.loads(rec.text)
            url = jd.get('tu')
            with open('pic_'+image_dir + str(ins) + '/' + url.split('tbn:')[1] + '.jpg', 'wb') as f:
                #print(image_dir + str(ins) + '/'url.split('tbn:')[1] + 'jpg')
                image_res = requests.get(url)
                f.write(image_res.content)               

In [ ]:
getImage('女人', 'PersonWC', 10)

## 使用CNN 分類抽煙或不抽煙！

In [ ]:

# Importing the Keras libraries and packages# Import 
from keras.models import Sequential 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D 
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

In [ ]:
# 1. Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64,
3), activation = 'relu'))

In [ ]:
# 2. Max Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
# 3. Flattening
classifier.add(Flatten())

In [ ]:
# 4. Fully Connection
classifier.add(Dense(units = 128, activation = 'relu')) 
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
classifier.compile(optimizer = 'adam', loss =
'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#?ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2, horizontal_flip = True)

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(
    'dataset/training_set/smo', target_size = (64, 64),
     batch_size = 64,
     class_mode = 'binary')

In [ ]:
test_set = test_datagen.flow_from_directory(
    'dataset/test_set/smo', target_size = (64, 64),
    batch_size = 64, 
    class_mode = 'binary')

In [ ]:
classifier.fit_generator(training_set, steps_per_epoch = 2000,
epochs = 300, validation_data = test_set, validation_steps = 105)

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/002.jpg', target_size= (64,64))
test_image

In [ ]:
test_image = image.img_to_array(test_image)

test_image

In [ ]:
type(test_image)

In [ ]:
test_image.shape

In [ ]:
test_image = np.expand_dims(test_image, axis = 0)

In [ ]:
a = np.array([[1,2,3],[4,5,6]])
a.shape

np.expand_dims(a, axis = 0)

In [ ]:
test_image.shape

In [ ]:
result = classifier.predict_classes(test_image)

In [ ]:
result